In [70]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle

In [71]:

#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

In [72]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

In [73]:
def get_non_nan_values_cols(p_df, col_name):
    
    slic_ind=p_df[col_name].isnull()
    list_ne=slic_ind.tolist()
    false_indices = [i for i in range(len(list_ne)) if not list_ne[i]]
    
    list_col_interest=p_df[col_name].to_list()
    
    non_values_list = []
    
    for false_ind in false_indices:
        non_val = list_col_interest[false_ind]
        non_values_list.append(non_val)
    
    return non_values_list


def get_file_names_from_ind_list(files_list, inds_list):
    
    selected_files_list=[files_list[sel_index] for sel_index in inds_list]
    
    return selected_files_list

In [74]:
def select_pickled_condition_files(path_pickled_files_list, cond_file_names_list):
    selected_file_cond_list = []
    
    for selected_file_path in path_pickled_files_list:
        dir_path, file_name=os.path.split(selected_file_path)
        sub_id = file_name[0:12]
    
        for cond_file in cond_file_names_list:
        
            if sub_id==cond_file:
            
                selected_file_cond_list.append(selected_file_path)
                #print(selected_file_path)
                
    return selected_file_cond_list

In [75]:
def min_max_norm(all_rr_one_list, all_rr_sublist):
    
    mean_norm_list =[]
    std_norm_list = []
    
    mean_norm_bpm_list = []
    std_norm_bpm_list = []
    
    all_rr_one_list = np.array(all_rr_one_list)
    max_val = np.max(all_rr_one_list)
    min_val =np.min(all_rr_one_list)
    
    range_list = max_val-min_val
    
    ####-------beats per minutes--------
    bpm_one_list = 60000/all_rr_one_list
    
    max_bpm = np.max(bpm_one_list)
    
    min_bpm = np.min(bpm_one_list)
    
    range_bpm =  max_bpm - min_bpm
    
    
    for sel_list in all_rr_sublist:
        
        sel_list_np = np.array(sel_list)
        
        
        
        sel_lis_np_sub = sel_list_np - min_val
        
        sel_lis_norm= sel_lis_np_sub/range_list
        
        mean_norm = np.mean(sel_lis_norm)
        mean_norm_list.append(mean_norm)
        
        std_norm  = np.std(sel_lis_norm)   
        std_norm_list.append(std_norm)
        
        ##------------for normalized BPM----------
        
        sel_list_bpm = 60000/sel_list_np
        
        
        
        sel_bpm_min_sub = sel_list_bpm - min_bpm
        
        sel_bpm_list_norm =  sel_bpm_min_sub/ range_bpm
        
        mean_norm_bpm = np.mean(sel_bpm_list_norm)
        
        std_norm_bpm = np.std(sel_bpm_list_norm)
        
        mean_norm_bpm_list.append(mean_norm_bpm)
        
        std_norm_bpm_list.append(std_norm_bpm)
        
        
        
        
        #print(mean_norm)
        #print(std_norm)
                
        
    return mean_norm_list, std_norm_list, mean_norm_bpm_list, std_norm_bpm_list

In [76]:
def split_bpm(mean_bpm_list):
    
    base_line = mean_bpm_list[0]
    tsst  = np.mean(mean_bpm_list[1:4])
    relax = np.mean(mean_bpm_list[4:])
    
    #tsst  = max(mean_bpm_list[1:4])
    #relax = min(mean_bpm_list[4:])

    new_array = [base_line,tsst, relax]
    
    return new_array

In [77]:
def get_cumsum_cond(input_array):
    
    mean_diff = np.diff(input_array)
    
    mean_diff = np.insert(mean_diff, [0], 0)
    
    mean_diff_cumsum = np.cumsum(mean_diff, dtype=float)
    
    return (mean_diff_cumsum)

In [78]:

def calc_rate_change_percent(cond_phy_array):
    
    prev_val = cond_phy_array[1]
    curr_val = cond_phy_array[2]
    change_val=((curr_val-prev_val)/abs(prev_val))
    change_percent = change_val*100
    #print(change_percent)
    
    return change_percent

In [80]:
def from_rr_to_time_anal(selected_path):
                 
    with open(selected_path, 'rb') as f:
        ibi_data = pickle.load(f)
                    
                    
    subject_id = ibi_data["subject_id"]
    event_interst = ibi_data["event_interst"]
    all_rr_one_list = ibi_data["all_rr_one_list"]
    all_rr_sublists = ibi_data["all_rr_sublists"]
            
    max_rr = np.max(all_rr_one_list)
    #print(max_rr)
    min_rr = np.min(all_rr_one_list)
    #print(min_rr)
    range_rr = max_rr - min_rr    
            
    mean_rr_cumm_list = []
    std_rr_cumm_list  = []
    mean_hr_list = []
    std_hr_list = []
            
    for index, selected_event in enumerate(event_interst):
        sel_rr = all_rr_sublists[index]
                
        #### heart rate from RR ineterval
        hr_from_rr = 60000/sel_rr
        mean_hr = np.mean(hr_from_rr)
        std_hr = np.std(hr_from_rr)
        mean_hr_list.append(mean_hr)
        std_hr_list.append(std_hr)
                
        ##### mean and std rr
        mean_rr = np.mean(sel_rr)     
        std_rr = np.std(sel_rr)
        mean_rr_cumm_list.append(mean_rr)
        std_rr_cumm_list.append(std_rr)
        #rmssd_event= 
                
                
    mean_norm_hrv_list, std_norm_hrv_list,  mean_norm_bpm_list, std_norm_bpm_list = min_max_norm(all_rr_one_list, all_rr_sublists)
    export_dict= {"subject_id":subject_id , 'event_interst': event_interst, "mean_bpm": mean_hr_list, "std_bpm":std_hr_list , 
                              "mean_ibi":mean_rr_cumm_list , "std_ibi": std_rr_cumm_list , "norm_mean_ibi":mean_norm_hrv_list,  "norm_std_ibi": std_norm_hrv_list,
                             "norm_mean_bpm":mean_norm_bpm_list, "norm_std_bpm":  std_norm_bpm_list}
            
    #export_dict= {"subject_id":subject_id , 'event_interst': event_interst, "mean_bpm": mean_hr_list, "std_bpm":std_hr_list , 
     #                         "mean_ibi":mean_rr_cumm_list , "std_ibi": std_rr_cumm_list , "norm_mean_ibi":mean_norm_hrv_list,  "norm_std_ibi": std_norm_hrv_list,
      #                       "norm_mean_bpm":mean_norm_bpm_list, "norm_std_bpm":  std_norm_bpm_list, "rmssd": ibi_data["rmssd"]}
            
            
    return export_dict

In [81]:
def filter_rr_stage_1(rr, throshold_high=0.5, threshold_low=0.5):
    
    #outlier_high = np.mean(rr) + throshold_high * np.std(rr)
    outlier_high =1200
    #print(outlier_high)
    
    rr_clean_high = [rr_val for rr_val in rr if rr_val <  outlier_high]
    
    rr_clean_high = np.array(rr_clean_high)
    
    #outlier_low = np.mean(rr_clean_high) - threshold_low*np.std(rr_clean_high)
    outlier_low = 400
    #print(outlier_low)
    
    rr_clean_high_low = [rr_val for rr_val in rr_clean_high if rr_val >  outlier_low]
    
    rr_clean_high_low= np.array(rr_clean_high_low)
    
    return rr_clean_high_low



def remove_outliers_rri(r_peaks_diff_msec, sampling_freq = 256, outlier_std = 2):
    
    # RR-intervals are the differences between successive peaks
    #r_peaks_sec = r_peaks*1/sampling_freq
    #r_peaks_msec = r_peaks_sec*1000

    #r_peaks_diff=np.diff(r_peaks_msec)
    
    rr_corrected = r_peaks_diff_msec.copy()

    rr_corrected[np.abs(zscore(r_peaks_diff_msec)) > outlier_std] = np.median(r_peaks_diff_msec)
    
    return rr_corrected



def from_rr_to_rr_interpolated(rr_msec, fs = 4):
    #create interpolation function based on the r-r samples
    
    # sample in seconds(x-axis)
    x = np.cumsum(rr_msec)/1200 
    f = interp1d(x, rr_msec, kind='cubic')
    
    steps = 1/fs
    
    # now we can sample from interpolation function
    xx = np.arange(1, np.max(x), steps)
    rr_interpolated_msec = f(xx)
    
    return rr_interpolated_msec

def frequency_domain_rr(rri, fs=4):
    # rri: rr interpolated
    # Estimate the spectral density using Welch's method
    fxx, pxx = signal.welch(x=rri, fs=fs)
    
    '''
    Segement found frequencies in the bands 
     - Very Low Frequency (VLF): 0-0.04Hz 
     - Low Frequency (LF): 0.04-0.15Hz 
     - High Frequency (HF): 0.15-0.4Hz
    '''
    cond_vlf = (fxx >= 0) & (fxx < 0.04)
    cond_lf = (fxx >= 0.04) & (fxx < 0.15)
    cond_hf = (fxx >= 0.15) & (fxx < 0.4)
    
    # calculate power in each band by integrating the spectral density 
    vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
    lf = trapz(pxx[cond_lf], fxx[cond_lf])
    hf = trapz(pxx[cond_hf], fxx[cond_hf])
    
    # sum these up to get total power
    total_power = vlf + lf + hf

    # find which frequency has the most power in each band
    peak_vlf = fxx[cond_vlf][np.argmax(pxx[cond_vlf])]
    peak_lf = fxx[cond_lf][np.argmax(pxx[cond_lf])]
    peak_hf = fxx[cond_hf][np.argmax(pxx[cond_hf])]

    # fraction of lf and hf
    lf_nu = 100 * lf / (lf + hf)
    hf_nu = 100 * hf / (lf + hf)
    
    results = {}
    results['Power VLF (ms2)'] = vlf
    results['Power LF (ms2)'] = lf
    results['Power HF (ms2)'] = hf   
    results['Power Total (ms2)'] = total_power

    results['LF/HF'] = (lf/hf)
    results['Peak VLF (Hz)'] = peak_vlf
    results['Peak LF (Hz)'] = peak_lf
    results['Peak HF (Hz)'] = peak_hf

    results['Fraction LF (nu)'] = lf_nu
    results['Fraction HF (nu)'] = hf_nu
    return results, fxx, pxx

def from_rr_to_freq_anal(selected_path):
                 
    with open(selected_path, 'rb') as f:
        ibi_data = pickle.load(f)
                    
                    
    subject_id = ibi_data["subject_id"]
    event_interst = ibi_data["event_interst"]
    all_rr_one_list = ibi_data["all_rr_one_list"]
    all_rr_sublists = ibi_data["all_rr_sublists"]
            
    max_rr = np.max(all_rr_one_list)
    #print(max_rr)
    min_rr = np.min(all_rr_one_list)
    #print(min_rr)
    range_rr = max_rr - min_rr    
            
    vlf_cumm_list = []
    lf_cumm_list  = []
    hf_cumm_list = []
    total_power_list = []
    #std_hr_list = []
            
    for index, selected_event in enumerate(event_interst):
        sel_rr = all_rr_sublists[index]
        rr_filtered_stage_1= filter_rr_stage_1(sel_rr)
        rr_filtered=remove_outliers_rri(rr_filtered_stage_1)
        interpolated_rr = from_rr_to_rr_interpolated(rr_filtered, fs = 4)
        
        print(np.max(interpolated_rr))
        results_freq, fxx, pxx =frequency_domain_rr(interpolated_rr)
        
        vlf=results_freq['Power VLF (ms2)']
        lf= results_freq['Power LF (ms2)']
        hf=results_freq['Power HF (ms2)']   
        total_power=results_freq['Power Total (ms2)']
        
                
        #### heart rate from RR ineterval
        #hr_from_rr = 60000/sel_rr
        #mean_hr = np.mean(hr_from_rr)
        #std_hr = np.std(hr_from_rr)
        #mean_hr_list.append(mean_hr)
        #std_hr_list.append(std_hr)
                
        ##### mean and std rr
        #mean_rr = np.mean(sel_rr)     
        #std_rr = np.std(sel_rr)
        #mean_rr_cumm_list.append(mean_rr)
        
        vlf_cumm_list.append(vlf)
        lf_cumm_list.append(lf)
        hf_cumm_list.append(hf)
        total_power_list.append(total_power)
                
                
    #mean_norm_hrv_list, std_norm_hrv_list,  mean_norm_bpm_list, std_norm_bpm_list = min_max_norm(all_rr_one_list, all_rr_sublists)
            
    export_dict= {"subject_id":subject_id , 'event_interst': event_interst, "vlf_power": vlf_cumm_list, "lf_power":lf_cumm_list , 
                              "hf_power":hf_cumm_list , "total_power": total_power_list}
            
            
    return export_dict

In [82]:
hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl"
#hrv_data_dir = "C:/Users/muhammad.saif/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/ecg_analysis/ecg_timadomain_pkl"


In [83]:
#dest_path = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_final_report/from_pkl_to_csv"
#dest_path = "D:/Datasets/data_sony_digiRelax/anal_freq_domain"

dest_path = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis"

#designated_folder = "hrv_analysis"
#comp_dest_path = os.path.join(dest_path, designated_folder)

isexist = os.path.exists(dest_path)

#if not isexist:
#    os.makedirs(comp_dest_path)
#    print("The new directory is created")

In [84]:
hrv_files_list = glob.glob(hrv_data_dir+ "/*.pkl")
hrv_files_list = sorted(hrv_files_list)

In [85]:
hrv_files_list

['/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl/VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317.pkl',
 '/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl/VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226.pkl',
 '/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl/VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415.pkl',
 '/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl/VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411.pkl',
 '/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timadomain_pkl/VP008_101123_DigiRelax_Experiment_2023-11-10_10h24.14.323.pkl',
 '/home/muhammad/Desktop/repos_ixp/tester_son

## for time domain analysis

In [86]:
for sel_file in hrv_files_list:
    fol_path, file_name_ii=os.path.split(sel_file)
    file_name_ii = file_name_ii[:-4]    
    hr_data_dict = from_rr_to_time_anal(sel_file)    
    csv_time_domain_feat_filename = file_name_ii + ".csv"
    dest_path_csv = os.path.join(dest_path,csv_time_domain_feat_filename)
    print(dest_path_csv)
    df_export = pd.DataFrame.from_dict(hr_data_dict)
    df_export.to_csv(dest_path_csv)
    
    
    
    

/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis/VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis/VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis/VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis/VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411.csv
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_timedomain_analysis/VP008_101123_DigiRelax_Experiment_2023-11-10_10h24.14.323.csv
/home/muhammad/Desktop/repos_ixp/tester_

In [67]:
dest_path_freq_doma= "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/ecg_eda_complete_analysis/ecg_analysis/ecg_freqdomain_anal"

#designated_folder = "hrv_analysis"
#comp_dest_path = os.path.join(dest_path, designated_folder)

isexist = os.path.exists(dest_path_freq_doma)

## For frequency domain analysis

In [68]:
for sel_file in hrv_files_list:
    fol_path, file_name_ii=os.path.split(sel_file)
    file_name_ii = file_name_ii[:-4]
    print(file_name_ii)
    hr_data_dict = from_rr_to_freq_anal(sel_file)
    
    csv_time_domain_feat_filename = file_name_ii + ".csv"
    dest_path_csv = os.path.join(dest_path_freq_doma,csv_time_domain_feat_filename)
    #print(dest_path_csv)
    df_export = pd.DataFrame.from_dict(hr_data_dict)
    df_export.to_csv(dest_path_csv)

VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317
1130.177720486806
1175.0275249474323
1159.6861886787235
1020.6004464714403
1203.1694264955356
1202.930141322579
1208.6925481943304
VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226
1065.811027082945
736.9621030996738
618.4478550156299
762.17973458561
1140.734008307893
1148.2850013965478
1143.545196556533
VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415
687.2576215510063
647.87144133489
611.1576595299197
658.4329151064312
777.9408335997806
754.4505147086809
748.2957559397905
VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411
1199.4851476216106
1179.1888117195515
1126.0505408686904
1059.1496483291528
1202.79880020834
1209.2353756190246
1205.5765657663133
VP008_101123_DigiRelax_Experiment_2023-11-10_10h24.14.323
1128.9769498336645
1002.9533407554186
923.3543732309624
998.2462646624026
1113.5751520063732
1053.4823513480737
1108.1957790192564
VP009_101123_DigiRelax_Experiment_2023-11-10_14h16.47.223
1024.39931

/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 130, using nperseg = 130
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 125, using nperseg = 125
  warnings.warn('nperseg = {0:d} is greater than input length '
/hom

1019.5016579583115
1044.463116449987
1200.8420687764867
1200.7312099400833
1216.6685754582802
VP030_291123_DigiRelax_Experiment_2023-11-29_09h54.53.886
893.3642901094684
686.4000323140714
581.292670234693
565.0616598519742
838.4690114212236
843.7682398472314
886.444528978684
VP031_291123_DigiRelax_Experiment_2023-11-29_14h41.15.470
1020.3357822668223
982.68561507693
843.7302968681428
846.1206335692519
1051.4999421951645
1047.2907661441657
1064.680482430779
VP032_301123_DigiRelax_Experiment_2023-11-30_09h49.51.787
907.0859024483419
602.4537017103416
617.5999269994687
554.2638382216568
823.4185543118878
820.2749430127475
808.231925374169
VP033_301123_DigiRelax_Experiment_2023-11-30_15h44.58.284
873.7963136204456
741.0356221506329
739.3526663826046
813.7977622718201
962.3421401577997
918.1333197697303
951.3626116420634
VP034_011223_DigiRelax_Experiment_2023-12-01_14h41.10.723
1028.0252894060154
806.0035586166696
727.9093692476923
690.269789184711
939.6403248497066
917.2799417321689
938.51